<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=82ef0b998e05135623c93360c3fcbf0cc51fb0820af22b7bcac33e2ea59ffc69
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-29 16:24:18
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -78.56 K (-0.56%)
Current PnL: -26.66 L (-17.22%)
CY Booked + Current PnL: -12.37 L (-7.99%)
-------------------
Total profit:  1.48 L
Total loss:  -28.14 L
-------------------
Total Booked + Current PnL: 14.61 L (11.43%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.29 L (65.02%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.12%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.93,-19.52,24.36,0.09,19758.0,-19668.0,81109.0,88.26,38.0,M-SC,2.93,253.0,-1.00,0.58,6.56,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.07,7.97,10.86,19.69,22482.0,15273.0,207015.0,-2.60,52.0,X-MC,2.38,78.0,0.68,1.48,15.30,XY25,NTT,AC
52,NESTLEIND,1377.00,0.58,12.84,7.58,21.40,22704.0,34094.0,299520.0,4.67,65.0,X-LC,5.53,12.0,1.50,2.14,19.99,XY25,NTT,FMCG
50,MASFIN,398.61,-1.04,-4.30,27.53,22.05,25814.0,-4215.0,93765.0,-17.34,47.0,H-SC,6.52,164.0,-0.16,0.67,36.29,XR,ATH,FINANCE
43,ITC,452.00,-0.38,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,48.0,X-LC,2.05,5.0,-0.11,1.70,3.84,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,6.37,-36.51,113.05,35.27,81505.0,-41453.0,72096.0,-51.71,58.0,L-SC,8.70,270.0,-0.51,0.52,76.46,XR,NTT,HOTELS
57,RAJOOENG,143.10,3.43,-37.66,123.94,39.61,79198.0,-38600.0,63900.0,-57.76,37.0,H-SC,31.73,135.0,-0.49,0.46,3.43,AR,ATH,MISC
44,JCHAC,2282.00,1.47,-34.82,53.40,-0.01,44485.0,-44499.0,83306.0,16943.48,42.0,M-SC,12.53,233.0,-1.00,0.60,6.62,OX40N,NTT,AC
74,TITAGARH,1548.00,1.10,-14.55,70.56,45.74,146012.0,-35244.0,206933.0,13.90,71.0,H-SC,5.82,174.0,-0.24,1.48,34.59,XY24,NTT,ENGINEERING
5,ASIANPAINT,3460.25,1.05,-2.99,24.68,20.95,60277.0,-7533.0,244235.0,-4.01,47.0,X-LC,14.83,31.0,-0.12,1.75,31.58,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
22,DIXON,18931.72,-3.82,-21.04,59.67,26.07,91976.0,-41080.0,154141.0,-61.61,25.0,X-MC,14.03,56.0,-0.45,1.10,0.00,X40N,ATH,IT
25,FINCABLES,1641.55,-3.26,-6.00,121.17,107.91,168173.0,-8853.0,138791.0,-23.39,41.0,M-SC,11.69,216.0,-0.05,0.99,3.49,OX40N,ATH,CABLES
48,KPIGREEN,730.66,-2.77,-18.58,80.48,46.95,82107.0,-23275.0,102022.0,-24.83,38.0,H-SC,10.07,154.0,-0.28,0.73,27.81,MH,ATH,POWER
13,BSOFT,831.70,-2.72,-20.85,90.52,50.79,100767.0,-29333.0,111320.0,1.66,58.0,H-SC,9.97,151.0,-0.29,0.80,29.61,XR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.07,-0.89,83.3,81.67,77450.0,-833.0,92977.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.05,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-3.26,-6.00,121.17,107.91,168173.0,-8853.0,138791.0,-23.39,41.0,M-SC,11.69,216.0,-0.05,0.99,3.49,OX40N,ATH,CABLES
77,TTKPRESTIG,770.00,-0.93,-19.52,24.36,0.09,19758.0,-19668.0,81109.0,88.26,38.0,M-SC,2.93,253.0,-1.00,0.58,6.56,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.05,-19.95,24.94,0.02,34850.0,-34823.0,139736.0,113.17,33.0,M-SC,9.82,234.0,-1.00,1.00,21.55,OX40N,NTT,PAINTS
47,KANSAINER,340.00,-1.24,-21.74,45.00,13.47,94964.0,-58635.0,211032.0,-67.92,54.0,H-SC,6.83,158.0,-0.62,1.51,7.30,XY24,NTT,PAINTS
66,SIS,528.00,-0.94,-22.79,57.05,21.26,48910.0,-25301.0,85731.0,2024.35,56.0,H-SC,4.76,166.0,-0.52,0.61,15.83,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.77,8.38,58.95,72.27,108260.0,14199.0,183647.0,-6.99,59.0,M-LC,4.43,99.0,0.13,1.31,15.07,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.77,8.38,58.95,72.27,108260.0,14199.0,183647.0,-6.99,59.0,M-LC,4.43,99.0,0.13,1.31,15.07,XR,NTT,IT
6,ASIANTILES,137.00,-2.07,-0.89,83.30,81.67,77450.0,-833.0,92977.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.05,XR,NTT,CERAMICS
38,INDIAMART,4810.62,-0.76,-5.49,118.73,106.72,138395.0,-6773.0,116563.0,-53.30,37.0,H-SC,5.05,139.0,-0.05,0.83,16.60,AR,ATH,MISC
25,FINCABLES,1641.55,-3.26,-6.00,121.17,107.91,168173.0,-8853.0,138791.0,-23.39,41.0,M-SC,11.69,216.0,-0.05,0.99,3.49,OX40N,ATH,CABLES
1,ABB,7934.00,-0.35,-5.24,53.61,45.57,132910.0,-13699.0,247920.0,-39.15,49.0,H-MC,4.53,120.0,-0.10,1.77,5.49,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-3.82,-21.04,59.67,26.07,91976.0,-41080.0,154141.0,-61.61,25.0,X-MC,14.03,56.0,-0.45,1.10,0.00,X40N,ATH,IT
9,BAJAJHFL,181.50,-1.01,-19.33,92.74,55.47,182002.0,-47036.0,196250.0,-28.46,26.0,X-MC,16.54,64.0,-0.26,1.40,0.56,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.05,-18.98,76.89,43.32,101464.0,-30912.0,131960.0,-30.40,29.0,X-SC,6.59,90.0,-0.30,0.94,1.32,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.58,-27.50,126.41,64.15,218083.0,-65431.0,172520.0,-57.62,31.0,X-SC,4.81,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
8,AWL,485.00,0.23,-24.83,104.21,53.50,236361.0,-74939.0,226812.0,-63.63,32.0,X-MC,6.81,58.0,-0.32,1.62,0.74,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.47,-2.98,24.86,21.14,54832.0,-6776.0,220563.0,-9.80,47.0,X-MC,1.22,74.0,-0.12,1.58,22.49,XY24,NTT,PAINTS
78,UNITDSPR,1644.0,0.09,5.02,14.99,20.77,36862.0,11764.0,245908.0,-4.14,51.0,X-MC,1.73,62.0,0.32,1.76,12.29,X40N,NTT,BREWERIES
15,CAMS,950.0,-1.48,-3.34,28.78,24.48,70062.0,-8415.0,243441.0,-81.54,40.0,X-SC,1.91,86.0,-0.12,1.74,19.02,X40N,NTT,MISC
65,SIEMENS,4671.5,-1.50,-17.94,52.95,25.51,80863.0,-33380.0,152715.0,-6.25,38.0,H-LC,1.94,49.0,-0.41,1.09,12.54,AR,ATH,ELECTRICAL
43,ITC,452.0,-0.38,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,48.0,X-LC,2.05,5.0,-0.11,1.70,3.84,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.58,-27.50,126.41,64.15,218083.0,-65431.0,172520.0,-57.62,31.0,X-SC,4.81,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.83,-21.40,80.03,41.51,165670.0,-56355.0,207010.0,-6.31,33.0,X-MC,8.16,61.0,-0.34,1.48,0.00,XY25,ATH,FMCG
22,DIXON,18931.72,-3.82,-21.04,59.67,26.07,91976.0,-41080.0,154141.0,-61.61,25.0,X-MC,14.03,56.0,-0.45,1.10,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,-1.03,-39.31,122.78,35.20,95880.0,-50579.0,78091.0,2.38,33.0,X-SC,17.26,92.0,-0.53,0.56,0.17,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-1.01,-19.33,92.74,55.47,182002.0,-47036.0,196250.0,-28.46,26.0,X-MC,16.54,64.0,-0.26,1.40,0.56,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.91,-29.12,101.53,42.84,46780.0,-18931.0,46075.0,-53.72,47.0,X-SC,33.01,83.0,-0.40,0.33,4.24,XY24,NTT,MISC
59,RELAXO,1176.00,0.02,-47.77,191.59,52.31,145264.0,-69340.0,75820.0,-44.34,43.0,X-SC,7.28,91.0,-0.48,0.54,1.49,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.03,-39.31,122.78,35.20,95880.0,-50579.0,78091.0,2.38,33.0,X-SC,17.26,92.0,-0.53,0.56,0.17,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.83,-3.69,28.83,24.08,35251.0,-4679.0,122271.0,-11.42,40.0,X-SC,6.98,89.0,-0.13,0.88,15.21,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.05,-18.98,76.89,43.32,101464.0,-30912.0,131960.0,-30.40,29.0,X-SC,6.59,90.0,-0.30,0.94,1.32,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.87,-10.60,35.01,20.71,118388.0,-40081.0,338156.0,-22.98,56.0,X-LC,3.00,1.0,-0.34,2.42,12.99,X40,ATH,IT
41,INFY,1972.00,-0.69,11.96,19.90,34.24,69714.0,37433.0,350321.0,-12.78,61.0,X-LC,6.73,2.0,0.54,2.51,21.24,X40,NTT,IT
75,TMPV,600.00,-0.06,-26.50,67.32,22.98,111773.0,-59866.0,166032.0,-24.30,49.0,X-LC,6.13,3.0,-0.54,1.19,4.43,XY24,NTT,AUTO
81,VBL,671.64,-0.03,-2.49,39.14,35.67,120537.0,-7879.0,307963.0,-14.05,57.0,X-LC,3.22,4.0,-0.07,2.20,11.01,X40N,ATH,FMCG
43,ITC,452.00,-0.38,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,48.0,X-LC,2.05,5.0,-0.11,1.70,3.84,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.0,-1.21,-38.01,114.39,32.89,54744.0,-29350.0,47857.0,-690.94,50.0,L-MC,6.76,259.0,-0.54,0.34,31.79,XR,NTT,BANKS
56,RAJESHEXPO,518.0,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.0,-2.07,-0.89,83.30,81.67,77450.0,-833.0,92977.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.05,XR,NTT,CERAMICS
13,BSOFT,831.7,-2.72,-20.85,90.52,50.79,100767.0,-29333.0,111320.0,1.66,58.0,H-SC,9.97,151.0,-0.29,0.80,29.61,XR,ATH,IT
69,SURYODAY,216.0,-2.61,-20.39,54.55,23.03,77764.0,-36516.0,142555.0,55.70,52.0,H-SC,6.86,167.0,-0.47,1.02,41.44,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.07,-0.89,83.30,81.67,77450.0,-833.0,92977.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.05,XR,NTT,CERAMICS
13,BSOFT,831.7,-2.72,-20.85,90.52,50.79,100767.0,-29333.0,111320.0,1.66,58.0,H-SC,9.97,151.0,-0.29,0.80,29.61,XR,ATH,IT
74,TITAGARH,1548.0,1.10,-14.55,70.56,45.74,146012.0,-35244.0,206933.0,13.90,71.0,H-SC,5.82,174.0,-0.24,1.48,34.59,XY24,NTT,ENGINEERING
56,RAJESHEXPO,518.0,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
84,WIPRO,420.0,-0.77,8.38,58.95,72.27,108260.0,14199.0,183647.0,-6.99,59.0,M-LC,4.43,99.0,0.13,1.31,15.07,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.97
2,50,76.61


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.89
MC    30.79
LC    24.28
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.90
X40      23.13
X40N     12.84
XR        9.82
XY25      9.20
AR        8.81
OX40N     7.54
SR        0.99
MH        0.73
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.22
H-SC    23.83
X-LC    20.82
M-SC    11.48
X-SC     8.01
H-MC     4.83
L-SC     1.57
M-MC     1.40
M-LC     1.31
H-LC     1.09
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.39,-7.30,41.80
IT,12.89,-16.59,75.22
FINANCE,9.63,-18.33,69.80
MISC,7.16,-29.80,82.90
ELECTRICAL,5.93,-13.11,54.54
PAINTS,5.84,-13.21,30.03
INSURANCE,4.71,-2.99,38.23
PHARMA,4.09,-2.83,35.20
AUTO,2.84,-32.12,77.92


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3162465.0,21
AR,1339515.0,10
XR,1290868.0,13
X40,991564.0,14
X40N,888392.0,9
OX40N,737604.0,10
XY25,354390.0,6
SR,282091.0,2
MH,82107.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3647519.0,25
M-SC,1455355.0,15
X-MC,1439931.0,16
X-LC,817320.0,11
X-SC,798689.0,8
H-MC,402982.0,3
L-SC,242658.0,3
M-LC,108260.0,1
H-LC,80863.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1251638.0      6
           AR         913470.0      5
M-SC       XY24       822811.0      6
H-SC       XR         790886.0      7
X-MC       X40        486576.0      7
           XY24       410346.0      3
           X40N       354857.0      4
H-SC       OX40N      327327.0      4
M-SC       OX40N      326574.0      5
X-LC       X40        323203.0      5
X-SC       X40N       316790.0      3
           XY24       300114.0      3
H-SC       SR         282091.0      2
H-MC       AR         217235.0      2
X-LC       X40N       216745.0      2
           XY24       191809.0      2
X-MC       XY25       188152.0      2
H-MC       XY24       185747.0      1
X-SC       X40        181785.0      2
M-SC       XR         178023.0      2
L-SC       XR         158955.0      2
M-SC       AR         127947.0      2
M-LC       XR         108260.0      1
X-LC       XY25        85563.0      2
L-SC       OX40N       83703.0      1
H-SC       MH          82107.0      1
H-LC       AR          80863.0      1
L-MC       XR          54744.0      1
M-MC       XY25        49530.0      1
L-LC       XY25        31145.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
